In [45]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
from nltk.corpus import stopwords
import re, string
import nltk

In [5]:
# sof=pd.read_csv("df_python_stackoverflow .csv")#.drop(columns='Unnamed: 0')
# reddit=pd.read_csv("df_python_reddit.csv")#[["question","answer"]]

In [111]:
# # concate two dataframes 
# df=pd.concat([sof,reddit]).drop(columns='Unnamed: 0')
# df=df.dropna(axis=0)
# df.head()

In [115]:
df=pd.read_csv("small_sample.csv")[['question','answer']].dropna(axis=0).reset_index().drop(columns='index')
df.head()

,question,answer
0,statistics analysis definition,Statistical analysis is the science of collect...
1,what is normal distribution,"In probability theory, a normal (or Gaussian o..."
2,Definition of statistical distributions,A statistical distribution is a representation...
3,Examples of Discrete Distributions,The Bernoulli Distribution\nThe Bernoulli dist...
4,Examples of Continuous Distributions,A normal distribution is the single most impor...


In [190]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r'[-()\"#/@;:<>{}`+=~|.!?,]\|', "", text)
    text=text.replace("[\'","").replace("\n"," ").replace("']"," ").replace('["',"").replace('"]',"").replace("it\'s","it's ").replace("\', \'","")
    text=text.replace("\',","").replace("it\'s","it is ").replace("it\\\'s","it is").replace(" \\"," ")
    text=text.replace('",'," ").replace("\',","").replace( ":\',","").replace("here\'s","").replace(":","").replace(',"',"")
    text=text.replace("[","").replace("]","").replace("\'s","'s")
    text=re.sub(r"let's", "let us", text)
    text = text.replace("\'s", "")

    
    return text

In [117]:
#declare answers and questions
questions=df.question
answers=df.answer
# Cleaning the questions
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
# Cleaning the answers
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [118]:
clean_questions[:10]

['statistics analysis definition',
 'what is normal distribution',
 'definition of statistical distributions',
 'examples of discrete distributions ',
 'examples of continuous distributions',
 'what is a probability mass function (pmf)?',
 'definition of  continuous variables',
 'what are continuous variables',
 'definition of discrete variable',
 'what is a discrete variable']

In [119]:
# list contains  punctuation
sw_list = stopwords.words('english')
sw_list += list(string.punctuation)
sw_list += ["''", '""', '...', '``', '’', '“', '’', '”', '‘', '‘',"'", '©',
'said',"'s", "also",'one',"n't",'com', 'satirewire', '-', '–','--' ,
'—', '_','satirewire.com']
sw_set = set(sw_list)

# tokenization
def process_data(string):
    tokens = nltk.word_tokenize(string) # tokenization
    stopwords_removed = [token.lower() for token in tokens if token.lower() not in sw_set]# stop workds removal
    return stopwords_removed

# Stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()
# create a function stemming() and loop through each word in a review
def stemming(string):
    stemmed_string=[]
    for w in string:
        stemmed_string.append(ps.stem(w))
    return stemmed_string

# import libraries
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# create a function  and loop through each word in  a review
def lemmatization(string):
    lemma_list=[]
    for word in string:
        lemma_word=lemmatizer.lemmatize(word,pos='v') 
        lemma_list.append(lemma_word)
    return lemma_list

# Conbime all functions above and obtian cleaned text data 
def data_preprocessing(text_data):
    #tokenization, stop words removal, punctuation marks removel
    processed_string=list(map(process_data,text_data))
    # stemming
    stemming_string=list(map(stemming,processed_string))
    # lemmatization
    lemma_string=list(map(lemmatization,stemming_string))
    
    return lemma_string

In [120]:
cleaned_questions=data_preprocessing(clean_questions)

In [203]:
# List of Greeting 
greeting=['hey', 'hi', 'hello', 'hey man', 'hi how are you', 'how’s it going', 
          'nice to meet you', 'how are you doing', 'what is up', 'what is new', 
          'what is going on', 'how is everything', 'how are things', 'how is life', 
          'how is your day', 'how is your day going', 'good to see you', 'nice to see you']
goodbye=["see you","bye","byebye","goodbye"]


In [ ]:
import random
print("=============# Hi There my name is Chacha. Let's Talk #=============")

while(True):
    
    
    question = input("You: ")
    if question.lower() in goodbye:
        print("Chacha: "+ "Bye")
        break 
    if  question.lower() in greeting:
        print("Chacha: "+ random.choice(greeting).title())
        continue
    # NLP
    cleaned_question=clean_text(question)
    processed_question=process_data(question)
    stemming_question=stemming(processed_question)
    lemma_question=lemmatization(stemming_question)
    # to find which row has intersection with the words from question you asked
    # which means to find the who have common elements between your question and the data
    inter_list=[]
    for i in cleaned_questions:
        if (set(lemma_question) & set(i)):
            inter_list.append((list(set(lemma_question) & set(i)),cleaned_questions.index(i)))
    # find the max length of common elements
    lengths=[len(inter_list[i][0]) for i in range(len(inter_list)) ]
    indexes=[]
    if len(lengths)>0:
        # find all the index whose correspondiong question data have the most common elements
        for i in range(len(inter_list)):
            if len(inter_list[i][0])==max(lengths):
                indexes.append(inter_list[i][1])  
             # to randomly find an answer based on the index
        answer_index=random.choice(indexes)
        print("Chacha: "+ answers[answer_index]+"\n")
    else:
        print("Sorry, I don't know. I need to learn more!")
        
    


=============# Hi There my name is Chacha. Let's Talk #=============
You: Hi
Chacha: Hey
You: what is statistical analysis
Chacha: Statistical analysis is the science of collecting data and uncovering patterns and trends. It’s really just another way of saying “statistics.” After collecting data you can analyze it to:
Summarize the data. For example, make a pie chart.
Find key measures of location. For example, the mean tells you what the average (or “middling”) number is in a set of data.
Calculate measures of spread: these tell you if your data is tightly clustered or more spread out. The standard deviation is one of the more commonly used measures of spread; it tells you how spread out your data is about the mean.
Make future predictions based on past behavior. This is especially useful in retail, manufacturing, banking, sports or for any organization where knowing future trends would be a benefit.
Test an experiment’s hypothesis. Collecting data from an experiment only tells a stor